I try to model a **two body skateboard** ( = a cart where the front and the rear axle can be steered and driven independently), which has a **joint** connecting the two parts of it\
the goal is to move it from a starting position to a final position, such that the centers of the skateboard bodies always follows **their** given line, the *street*s\
**opty** can steer the axles and drive them.\
Basically this was to try *holonomic* constraints with *opty*\
The variables are defined in the code below.

I believe, this is a difficult problem in the 'real world', so not surprising to me, that whether a solution can be obtained very much depends of the shape of the curves on on how I iterate towards the final curve. 

In [ ]:
import sympy.physics.mechanics as me
from collections import OrderedDict
import time
import numpy as np
import sympy as sm

from scipy.optimize import fsolve
from scipy.interpolate import CubicSpline

from opty.direct_collocation import Problem
from opty.utils import parse_free

import matplotlib.pyplot as plt
import matplotlib
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib.animation import FuncAnimation

## Timo's function
to create *obj* and  *obj_gradient* functions as needed with opty. 

In [ ]:
def create_objective_function(
    objective, state_symbols, input_symbols, unknown_symbols,
    N, node_time_interval=1.0, integration_method="backward euler"):
    """Creates function to evaluate the objective and objective gradient.

    Parameters
    ----------
    objective : sympy.Expr
        The objective function to be minimized, which is a function of the
        states and inputs. The objective function can contain non-nested
        indefinite integrals of time, e.g. ``Integral(f(t)**2, t)``.
    state_symbols : iterable of symbols
        The state variables.
    input_symbols : iterable of symbols
        The input variables.
    unknown_symbols : iterable of symbols
        The unknown parameters.
    N : int
        Number of collocation nodes, i.e. the number of time steps.
    node_time_interval : float
        The value of the time interval. The default is 1.0, as this term only
        appears in the objective function as a scaling factor.
    integration_method : str, optional
        The method used to integrate the system. The default is "backward
        euler".

    """
    def lambdify_function(expr, multiplication_array, take_sum):
        if take_sum:
            def integration_function(x):
                return node_time_interval * np.sum(x * multiplication_array)
        else:
            def integration_function(x):
                return node_time_interval * x * multiplication_array
        return sm.lambdify(
            (states, inputs, params), expr,
            modules=[{int_placeholder.name: integration_function}, "numpy"],
            cse=True)

    def parse_expr(expr, in_integral=False):
        if not expr.args:
            return expr
        if isinstance(expr, sm.Integral):
            if in_integral:
                raise NotImplementedError("Nested integrals are not supported.")
            if expr.limits != ((me.dynamicsymbols._t,),):
                raise NotImplementedError(
                    "Only indefinite integrals of time are supported.")
            return int_placeholder(parse_expr(expr.function, True))
        return expr.func(*(parse_expr(arg) for arg in expr.args))

    # Parse function arguments
    states = sm.ImmutableMatrix(state_symbols)
    inputs = sm.ImmutableMatrix(sort_sympy(input_symbols))
    params = sm.ImmutableMatrix(sort_sympy(unknown_symbols))
    if states.shape[1] > 1 or inputs.shape[1] > 1 or params.shape[1] > 1:
        raise ValueError(
            'The state, input, and unknown symbols must be column matrices.')
    n, q = states.shape[0], inputs.shape[0]
    i_idx, r_idx = n * N, (n + q) * N

    # Compute analytical gradient of the objective function
    objective_grad = sm.ImmutableMatrix([objective]).jacobian(
        states[:] + inputs[:] + params[:])

    # Replace the integral with a custom function
    int_placeholder = sm.Function("_IntegralFunction")
    objective = parse_expr(objective)
    objective_grad = tuple(parse_expr(objective_grad))

    # Replace zeros with an array of zeros, otherwise lambdify will return a
    # scalar zero instead of an array of zeros.
    objective_grad = tuple(
        np.zeros(N) if grad == 0 else grad for grad in objective_grad[:n + q]
        ) + tuple(objective_grad[n + q:])

    # Define evaluation functions based on the integration method
    if integration_method == "backward euler":
        obj_expr_eval = lambdify_function(
            objective, np.hstack((0, np.ones(N - 1))), True)
        obj_grad_time_expr_eval = lambdify_function(
            objective_grad[:n + q], np.hstack((0, np.ones(N - 1))), False)
        obj_grad_param_expr_eval = lambdify_function(
            objective_grad[n + q:], np.hstack((0, np.ones(N - 1))), True)
        def obj(free):
            states = free[:i_idx].reshape((n, N))
            inputs = free[i_idx:r_idx].reshape((q, N))
            return obj_expr_eval(states, inputs, free[r_idx:])

        def obj_grad(free):
            states = free[:i_idx].reshape((n, N))
            inputs = free[i_idx:r_idx].reshape((q, N))
            return np.hstack((
                *obj_grad_time_expr_eval(states, inputs, free[r_idx:]),
                obj_grad_param_expr_eval(states, inputs, free[r_idx:])
             ))

    elif integration_method == "midpoint":
        obj_expr_eval = lambdify_function(
            objective, np.ones(N - 1), True)
        obj_grad_time_expr_eval = lambdify_function(
            objective_grad[:n + q], np.hstack((0.5, np.ones(N - 2), 0.5)),
            False)
        obj_grad_param_expr_eval = lambdify_function(
            objective_grad[n + q:], np.ones(N - 1), True)
        def obj(free):
            states = free[:i_idx].reshape((n, N))
            states_mid = 0.5 * (states[:, :-1] + states[:, 1:])
            inputs = free[i_idx:r_idx].reshape((q, N))
            inputs_mid = 0.5 * (inputs[:, :-1] + inputs[:, 1:])
            return obj_expr_eval(states_mid, inputs_mid, free[r_idx:])

        def obj_grad(free):
            states = free[:i_idx].reshape((n, N))
            states_mid = 0.5 * (states[:, :-1] + states[:, 1:])
            inputs = free[i_idx:r_idx].reshape((q, N))
            inputs_mid = 0.5 * (inputs[:, :-1] + inputs[:, 1:])
            return np.hstack((
                *obj_grad_time_expr_eval(states, inputs, free[r_idx:]),
                obj_grad_param_expr_eval(states_mid, inputs_mid, free[r_idx:])
            ))

    else:
        raise NotImplementedError(
            f"Integration method '{integration_method}' is not implemented.")

    return obj, obj_grad


def sort_sympy(seq):
    """Returns a sorted list of the symbols."""
    seq = list(seq)
    try:  # symbols
        seq.sort(key=lambda x: x.name)
    except AttributeError:  # functions
        seq.sort(key=lambda x: x.__class__.__name__)
    return seq

Defines the shape of the two streets (strasse = German for street)

In [ ]:
def strasse(x, a, b):
    return a * (sm.sin(b * x) + sm.cos(3 * b * x))

def strasse1(x, a, b):
    return a * sm.sin(b * x) + sm.cos(4 * b * x) + 1.

## Kane's equations of motion
Note, that the holonomic constraints is added, to give $EOM = \begin{pmatrix} \text{kinematic constraints} \\ fr + fr^{\star} \\ \text{holonomic constraints} \end{pmatrix} $ as this is the form opty needs it in.\
otherwise nothing special.

In [ ]:
start = time.time()

N, A0, A1, Ab, Af = sm.symbols('N A0 A1 Ab Af', cls= me.ReferenceFrame)
t = me.dynamicsymbols._t
O, Pb, Dmc, P1, Dmc2, Pf = sm.symbols('O Pb Dmc P1 Dmc2 Pf', cls= me.Point)
O.set_vel(N, 0)

q0, q1, qb, qf     = me.dynamicsymbols('q_0 q_1 q_b q_f')                     # gen. coordinates of the main body, the rear axle, the front axle
u0, u1, ub, uf     = me.dynamicsymbols('u_0 u_1 u_b u_f')                     # their speeds
x, y           = me.dynamicsymbols('x y')                                     # coordinates of the point, where the rear axle attaches to the main body
ux, uy         = me.dynamicsymbols('u_x u_y')                                 # their speeds
Tb, Tf, Fb, Ff = me.dynamicsymbols('T_b T_f F_b F_f')                         # torque at back wheel, torque at front wheel, forces on Pb, Pf. Control variables

l, m0, mb, mf, iZZ0, iZZb, iZZf = sm.symbols('l m0 mb mf iZZ0, iZZb, iZZf')   # link length, mass, inertia of skateboard
a, b = sm.symbols('a b')                                                      # strasse parameters

A0.orient_axis(N, q0, N.z)
A0.set_ang_vel(N, u0 * N.z)
A1.orient_axis(N, q1, N.z)
A1.set_ang_vel(N, u1 * N.z)
Ab.orient_axis(N, qb, N.z)
Ab.set_ang_vel(N, ub * N.z)
Af.orient_axis(N, qf, N.z)
Af.set_ang_vel(N, uf * N.z)

Pb.set_pos(O, x * N.x + y * N.y)
Pb.set_vel(N, ux * N.x + uy * N.y)

Dmc.set_pos(Pb, l/2 * A0.y)
Dmc.v2pt_theory(Pb, N, A0)

P1.set_pos(Pb, l * A0.y)
P1.v2pt_theory(Pb, N, A0)

Dmc2.set_pos(P1, l/2 * A1.y)
Dmc2.v2pt_theory(P1, N, A1)

Pf.set_pos(P1, l * A1.y)   
Pf.v2pt_theory(P1, N, A1) 

constr_Dmc    = me.dot(Dmc.pos_from(O), N.y) - strasse(me.dot(Dmc.pos_from(O), N.x), a, b)            # Dmc must always be on the strasse
constr_Dmc_dt = constr_Dmc.diff(t)                                                                    # resulting speed constraint.

constr_Dmc2    = me.dot(Dmc2.pos_from(O), N.y) - strasse1(me.dot(Dmc2.pos_from(O), N.x), a, b)         # Dmc2 must always be on the strasse
constr_Dmc2_dt = constr_Dmc2.diff(t)                                                                   # resulting speed constraint.

# Create the skateboard body
I0 = me.inertia(A0, 0, 0, iZZ0)
body0 = me.RigidBody('body0', Dmc, A0, m0, (I0, Dmc))
I1 = me.inertia(A1, 0, 0, iZZ0)
body1 = me.RigidBody('body1', P1, A1, m0, (I1, P1))
Ib = me.inertia(Ab, 0, 0, iZZb)
bodyb = me.RigidBody('bodyb', Pb, Ab, mb, (Ib, Pb))
If = me.inertia(Af, 0, 0, iZZf)
bodyf = me.RigidBody('bodyf', Pf, Af, mf, (If, Pf))
BODY = [body0, body1, bodyb, bodyf]


# Forces
FL = [(Pb, Fb * Ab.y), (Pf, Ff * Af.y), (Ab, Tb * N.z), (Af, Tf * N.z)]

kd = sm.Matrix([ux - x.diff(t), uy - y.diff(t), u0 - q0.diff(t), u1 - q1.diff(t), ub - qb.diff(t), uf - qf.diff(t)]) # kinematic differential equations
speed_constr = [constr_Dmc_dt, constr_Dmc2_dt]                                                                       # speed constraints
hol_constr = sm.Matrix([constr_Dmc, constr_Dmc2])                                                                    # holonomic constraints: Dmc must always be on the strasse

q_ind = [x, y, qb, qf]
q_dep = [q0, q1]
u_ind = [ux, uy, ub, uf]
u_dep = [u0, u1]

# Create the KanesMethod object
KM = me.KanesMethod(
                    N, q_ind=q_ind, u_ind=u_ind, 
                    kd_eqs=kd, 
                    q_dependent=q_dep, 
                    u_dependent=u_dep, 
                    configuration_constraints=hol_constr, 
                    velocity_constraints=speed_constr,
                    )
(fr, frstar) = KM.kanes_equations(BODY, FL)

EOM = kd.col_join(fr + frstar) 
EOM = EOM.col_join(hol_constr)

print('EOM shape', EOM.shape, '\n')
print('EOM DS', me.find_dynamicsymbols(EOM), '\n')
print('EOM FS', EOM.free_symbols)
print(f'EOMs contain {sm.count_ops(EOM)} operations, {sm.count_ops(sm.cse(EOM))} after cse')

# needed further down
strasse2 = strasse(x, a, b)
strasse3 = strasse1(x, a, b)
strasse_lam = sm.lambdify((x, a, b), strasse2, cse=True)
strasse1_lam = sm.lambdify((x, a, b), strasse3, cse=True)
constr_Dmc_lam = sm.lambdify((y, x, q0, a, b, l), constr_Dmc, cse=True)                                 # meeded to ensure the configurtation constraint is satisfied when the optimisation starts
constr_Dmc2_lam = sm.lambdify((q1, x, y, q0, a, b, l), constr_Dmc2, cse=True)                           # meeded to ensure the configurtation constraint is satisfied when the optimisation starts

Set up the machinery for **opty**

In [ ]:
state_symbols     = tuple((x, y, q0, q1, qb, qf, ux, uy, u0, u1, ub, uf))
laenge            = len(state_symbols)
constant_symbols  = (a, b, l, m0, mb, mf, iZZ0, iZZb, iZZf)
specified_symbols = (Fb, Ff, Tb, Tf)
unknown_symbols   = []

#======================================================================================================
methode = 'backward euler'  # The integration method to use. backward euler or midpoint are the choices
plot_initial   = True         # True if the initial guess should be plotted
opty_info      = True         # True if the opty info will be printed
Timo_objective = True       # True if the objective function will be created with Timo's function
#======================================================================================================

duration  = 7.5                                             # The duration of the simulation.
num_nodes =  300                                            # The number of nodes to use in the direct collocation problem.
t0, tf    = 0.0, duration                                   # The initial and final times

initial_guess = np.ones((len(state_symbols) + len(specified_symbols)) * num_nodes) * 0.01


interval_value = duration / (num_nodes - 1)

# Specify the known system parameters.
par_map = OrderedDict()
par_map[m0]       = 1.0                                      # Mass of the block
par_map[mb]       = 0.1                                      # Mass of the back wheel
par_map[mf]       = 0.1                                      # Mass of the front wheel
par_map[iZZ0]     = 1.                                       # Inertia of the block
par_map[iZZb]     = 0.1                                      # Inertia of the back wheel
par_map[iZZf]     = 0.1                                      # Inertia of the front wheel
par_map[l]        = 3.0                                      # Length of the skateboard
par_map[a]        = 1.5                                      # Parameter of the street
par_map[b]        = 0.                                       # Parameter of the street   

x1                = 0.0                                      # Initial x position of Pb
q01               = -0.5                                     # Initial angle of the first link

# Calculate the initial value of y, so that the configuration constraint is satisfied.
# as the initial speeds are always zero, the resulting speed constraint is always satisfied
def hol_func(x0, *args):
    return constr_Dmc_lam(x0, *args)

def hol_func1(x0, *args):
    return constr_Dmc2_lam(x0, *args)

x0 = 1.                                                      # initial guess for the fsolve for y1
args = (x1, q01, par_map[a], par_map[b], par_map[l])
y1   = fsolve(hol_func, x0, args)
    
args = (x1, y1[0], q01, par_map[a], par_map[b], par_map[l])
q11   = fsolve(hol_func1, x0, args)    

if Timo_objective == True:  
# Create the objective function
    objective = sm.Integral((Fb**2 ) + (Ff**2) + (Tb**2) + (Tf**2)) * 0.00001
# specify the objective function and it's gradient
    obj, obj_grad = create_objective_function( 
        objective, 
        state_symbols, 
        specified_symbols, 
        unknown_symbols,
        num_nodes, 
        interval_value, 
        methode)
else:
# Specify the objective function and it's gradient.
    def obj(free): 
#        Minimize the sum of the squares of the controls."""
        Fx = free[laenge * num_nodes: (laenge + 1) * num_nodes]
        Fy = free[(laenge + 1) * num_nodes: (laenge + 2) * num_nodes]
        Tb = free[(laenge + 2) * num_nodes: (laenge + 3) * num_nodes]
        Tf = free[(laenge + 3) * num_nodes: (laenge + 4) * num_nodes]   
    
        return interval_value * (np.sum(Fx**2) + np.sum(Fy**2) + np.sum(Tb**2) + np.sum(Tf**2))

    def obj_grad(free):
        grad = np.zeros_like(free)
        grad[laenge * num_nodes : ]       = 2.0 * interval_value * free[laenge * num_nodes :]
        return grad

initial_state_constraints = {
                            x: x1,
                            y: y1[0],
                            q0: q01,
                            q1: q11[0],
                            qb: -0.2,
                            qf: -0.3,
                            ux: 0.,
                            uy: 0.,
                            u0: 0.,
                            u1: 0., 
                            ub: 0.,
                            uf: 0.,
                            }


final_state_constraints    = {
                             x: 12.,
                             ux: 0.,
                             }    
    
instance_constraints = tuple(xi.subs({t: t0}) - xi_val for xi, xi_val in initial_state_constraints.items()) + tuple(xi.subs({t: tf}) - xi_val for xi, xi_val in final_state_constraints.items())
print('instance constraints:', instance_constraints, '\n')

grenze = 3.
bounds = {
         Fb: (-grenze, grenze),   
         Ff: (-grenze, grenze),
         Tb: (-grenze, grenze),
         Tf: (-grenze, grenze),
        }



#======================================================================================================
#======================================================================================================
if plot_initial:
    Pbl, Pbr, Pfl, Pfr = sm.symbols('Pbl Pbr Pfl Pfr', cls= me.Point)
    la = sm.symbols('la')                                                               # length of the axles
    fb, ff = sm.symbols('f_b f_f')                                                      # forces at the axles

    Pbl.set_pos(Pb, -la/2 * Ab.x)
    Pbr.set_pos(Pb, la/2 * Ab.x)
    Pfl.set_pos(Pf, -la/2 * Af.x)
    Pfr.set_pos(Pf, la/2 * Af.x)

    coordinates = Pb.pos_from(O).to_matrix(N)
    for point in (Dmc, P1, Dmc2, Pf, Pbl, Pbr, Pfl, Pfr):
        coordinates = coordinates.row_join(point.pos_from(O).to_matrix(N))
    coordinates_lam = sm.lambdify((x, y, q0, q1, qb, qf, l, a, b, la), coordinates, cse=True)

    pL_vals = [par_map[i] for i in (l, a, b)] + [par_map[l] / 2]
    y0 = [initial_state_constraints[i] for i in (x, y, q0, q1, qb, qf)]
    coord_werte = coordinates_lam(*y0, *pL_vals)

    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.set_aspect('equal')
    ax.grid(True)

    strasse_x = np.linspace(-3, 3, 100)
    ax.plot(strasse_x, strasse_lam(strasse_x, par_map[a], par_map[b]), color='black', linestyle='-', linewidth=0.5)   # the first street
    ax.plot(strasse_x, strasse1_lam(strasse_x, par_map[a], par_map[b]), color='red', linestyle='-', linewidth=0.5)    # the second street
    ax.plot(coord_werte[0, :], coord_werte[1, :], 'o', color='blue', markersize=5)                                    # the points

    ax.plot([coord_werte[0, 0], coord_werte[0, 2]], [coord_werte[1, 0], coord_werte[1, 2]], color='blue', linestyle='-', linewidth=1.)  # the first link
    ax.plot([coord_werte[0, 2], coord_werte[0, 4]], [coord_werte[1, 2], coord_werte[1, 4]], color='blue', linestyle='-', linewidth=1.)  # the second link
    ax.plot([coord_werte[0, -1], coord_werte[0, -2]], [coord_werte[1, -1], coord_werte[1, -2]], color='magenta', linestyle='-', linewidth=1.)  # the front axle
    ax.plot([coord_werte[0, -3], coord_werte[0, -4]], [coord_werte[1, -3], coord_werte[1, -4]], color='red', linestyle='-', linewidth=1.)  # the back axle
    ax.set_title('The initial configuration with the initial shape of the lines')
    
else:
   pass

**solve** the optimization problem.

In [ ]:
solution_list = []
b_parameter   = []
g_list        = []
# Here I iterate, making the streets succesively more uneven.
# I use the previous solution as the initial guess for the next iteration.
inkrement = 0.05
for i in range(6):
    par_map[b] = inkrement * i
    b_parameter.append(par_map[b])
    prob = Problem(obj, obj_grad, EOM, state_symbols, num_nodes, interval_value,
        known_parameter_map=par_map,
        instance_constraints=instance_constraints,
#        bounds=bounds,
        integration_method=methode)

    prob.add_option('max_iter', 3000)           # default is 3000
# Find the optimal solution.
    solution, info = prob.solve(initial_guess)
    solution_list.append(solution)
    print(f'{i+1} - th iteration')
    if opty_info:
        print('message from optimizer:', info['status_msg'])
        print('Iterations needed',len(prob.obj_value))
        print(f"objective value {info['obj_val']:.3e} \n")
#     prob.plot_objective_value()
    initial_guess = solution
    g_list.append(info['g'])

Plot the **Errors** in the variables contained in the state symbols\
The results of the iterations (loops) are stored. With *welche* you may select which one will be plotted.

In [ ]:
welche = -1
fehler = g_list[welche]
anzahl = len(state_symbols)
fig, ax = plt.subplots(anzahl, 1, figsize=(10, 2.25*anzahl), sharex=True)
times = np.linspace(0.0, duration, num=num_nodes)

for i, j in enumerate(state_symbols):
    if j in final_state_constraints.keys():
        farbe1 = 'red'
    else:
        farbe1 = 'black'

    if i < laenge:
        farbe = 'b'
    else:
        farbe = 'r'
    ax[i].plot(times, fehler[i * num_nodes:(i + 1) * num_nodes], color = farbe, lw=0.5)
    ax[i].grid()
    ax[i].set_ylabel(f'${str(j)}$', color=farbe1)
ax[-1].set_xlabel('time')
ax[0]. set_title(f'Errors in the generalized coordinates and speeds \n coordinates/speeds in the final_state_constraints are red', color = 'b');
print(f'error in the final state constraints x:  {fehler[-2]:.3e},\n error in the final state constrains ux: {fehler[-1]:.3e}')


Plot the state variables and the forces / torques
Coordinates in the *final_state_constraints* dictionary are labelled in red.

In [ ]:
solution = solution_list[welche]
state_vals, input_vals, _ = parse_free(solution, len(state_symbols), len(specified_symbols), num_nodes)
anzahl = len(state_symbols) + len(specified_symbols)
fig, ax = plt.subplots(anzahl, 1, figsize=(10, 2.25*anzahl), sharex=True)
times = np.linspace(0.0, duration, num=num_nodes)

for i, j in enumerate(state_symbols):
    if j in final_state_constraints.keys():
        farbe1 = 'red'
    else:
        farbe1 = 'black'
    ax[i].plot(times, state_vals[i], lw=0.5)
    ax[i].set_ylabel(f'${str(j)}$', color=farbe1)
    ax[i].grid()
ax[0]. set_title('Generalized coordinates and speeds', color = 'b')

for i, j in enumerate(specified_symbols):
    ax[len(state_symbols) + i].plot(times, input_vals[i], lw=0.5, color='r')
    ax[-1].set_xlabel('time')
    ax[len(state_symbols) + i].set_ylabel(f'${str(j)}$')
    ax[len(state_symbols) + i].grid()
    ax[len(state_symbols)].set_title('Control force(s) / torque(s)', color = 'r');

aminate the **skateboard**\
Nothing special, just a lot of points have to be set, this is why it looks so large.\
The green arrows symbolize the forces which opty calculated to drive the scateboard. They are perpendicular to the axles.\
The center points of each body should stay on their respective lines, that is the red dot on the red line, the black dot on the black line. 

In [ ]:
fps = 30

def add_point_to_data(line, x, y):
# this function draws a line to trace the point(x|y)
    old_x, old_y = line.get_data()
    line.set_data(np.append(old_x, x), np.append(old_y, y))


solution = solution_list[welche]
par_map[b] = b_parameter[welche]

# copied from Timo/Jason!
# the CubicSpline object is used to interpolate the solution at any point in time
state_vals, input_vals, _ = parse_free(solution, len(state_symbols), len(specified_symbols), num_nodes)
t_arr = np.linspace(t0, tf, num_nodes)
state_sol = CubicSpline(t_arr, state_vals.T)
input_sol = CubicSpline(t_arr, input_vals.T)

# create additional points for the axles
Pbl, Pbr, Pfl, Pfr = sm.symbols('Pbl Pbr Pfl Pfr', cls= me.Point)
Fbq, Ffq = sm.symbols('Fbq Ffq', cls=me.Point)                                      # end points of the forces
la = sm.symbols('la')                                                               # length of the axles
fb, ff = sm.symbols('f_b f_f')                                                      # forces at the axles

Pbl.set_pos(Pb, -la/2 * Ab.x)                                                       # the back left wheel
Pbr.set_pos(Pb, la/2 * Ab.x)                                                        # the back right wheel                                                              
Pfl.set_pos(Pf, -la/2 * Af.x)                                                       # the front left wheel
Pfr.set_pos(Pf, la/2 * Af.x)                                                        # the front right wheel

Fbq.set_pos(Pb, fb * Ab.y)                                                          # tip of the force at the back axle
Ffq.set_pos(Pf, ff * Af.y)                                                          # tip of the force at the front axle

coordinates = Pb.pos_from(O).to_matrix(N)
for point in (Dmc, P1, Dmc2, Pf, Pbl, Pbr, Pfl, Pfr, Fbq, Ffq):
    coordinates = coordinates.row_join(point.pos_from(O).to_matrix(N))
coordinates_lam = sm.lambdify((x, y, q0, q1, qb, qf, fb, ff, l, a, b, la), coordinates, cse=True)

l1 = par_map[l]
a1 = par_map[a]
b1 = par_map[b]
la1 = l1 / 2

xmin = -2.5
xmax = 12.5
ymin = -2.5
ymax = 6.
fig = plt.figure(figsize=(9, 9))
ax  = fig.add_subplot(111)
ax.set_xlim(xmin-1, xmax + 1.)
ax.set_ylim(ymin-1, ymax + 1.)
ax.set_aspect('equal')
ax.grid()

strasse_x = np.linspace(xmin, xmax, 100)
ax.plot(strasse_x, strasse_lam(strasse_x, par_map[a], par_map[b]), color='black', linestyle='-', linewidth=0.75) # the first street
ax.plot(strasse_x, strasse1_lam(strasse_x, par_map[a], par_map[b]), color='red', linestyle='-', linewidth=0.75)  # the second street

ax.axvline(initial_state_constraints[x], color='r', linestyle='--', linewidth=1)                                # the initial position
ax.axvline(final_state_constraints[x], color='green', linestyle='--', linewidth=1)                              # the final position

# Initialize the skateboard objects.
line1,  = ax.plot([], [], color='blue', lw=2)                                                                    # the skateboard first body
line1a, = ax.plot([], [], color='blue', lw=2)                                                                    # the skateboard, second body
line2,  = ax.plot([], [], color='red', lw=2)                                                                     # the back axle
line3,  = ax.plot([], [], color='magenta', lw=2)                                                                 # the front axle
line4   = ax.quiver([], [], [], [], color='green', scale=7, width=0.004)                                         # the force at the back axle
line5   = ax.quiver([], [], [], [], color='green', scale=7, width=0.004)                                         # the force at the front axle
line6,  = ax.plot([], [], color='blue', marker ='o', markersize=7)                                               # P1, the joint between the two bodies    
line7,  = ax.plot([], [], color='black', marker ='o', markersize=7)                                              # Dmc, center of first body    
line8,  = ax.plot([], [], color='red', marker ='o', markersize=7)                                                # Dmc2, center of second body
line9,  = ax.plot([], [], lw=0.5, linestyle ='-.', color='red')                                              # Pf, the joint between the two bodies
line10, = ax.plot([], [], lw=0.5, linestyle ='-.', color='magenta')                                              # Pb, the joint between the two bodies

# Function to update the plot for each animation frame
def update(frame):
    message = (f'running time {frame:.2f} sec \n the back axle is red, the front axle is magenta \n The driving forces are green')
    ax.set_title(message, fontsize=12)
# Pb, Dmc, P1, Dmc2, Pf, Pbl, Pbr, Pfl, Pfr, Fbq, Ffq                                                           sequence of the points in coords
    coords = coordinates_lam(*state_sol(frame)[: 6], *input_sol(frame)[0: 2],  l1, a1, b1, la1)
    line1.set_data([coords[0, 0], coords[0, 2]], [coords[1, 0], coords[1, 2]])                                      # update first body
    line1a.set_data([coords[0, 2], coords[0, 4]], [coords[1, 2], coords[1, 4]])                                     # update second bpdy
    line2.set_data([coords[0, 5], coords[0, 6]], [coords[1, 5], coords[1, 6]])                                      # update back axle
    line3.set_data([coords[0, 7], coords[0, 8]], [coords[1, 7], coords[1, 8]])                                      # update front axle

    line4.set_offsets([coords[0, 0], coords[1, 0]])                                                                 # update force at the back axle
    line4.set_UVC(coords[0, -2] - coords[0, 0] , coords[1, -2] - coords[1, 0])

    line5.set_offsets([coords[0, 4], coords[1, 4]])
    line5.set_UVC(coords[0, -1] - coords[0, 4] , coords[1, -1] - coords[1, 4])

    line6.set_data([coords[0, 2]], [coords[1, 2]])                                                                   # update P1
    line7.set_data([coords[0, 1]], [coords[1, 1]])                                                                   # update Dmc
    line8.set_data([coords[0, 3]], [coords[1, 3]])                                                                   # update Dmc2

    add_point_to_data(line9, coords[0, 0], coords[1, 0])                                                            # trace Pb
    add_point_to_data(line10, coords[0, 4], coords[1, 4])                                                           # trace Pf

    return line1, line1a, line2, line3, line4, line5, line6, line7, line8, line9, line10

# Set labels and legend
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

# Create the animation
animation = FuncAnimation(fig, update, frames = np.arange(t0, tf, 1 / fps), interval=50, blit=False)
plt.close(fig)  # Prevents the final image from being displayed directly below the animation
# Show the plot
display(HTML(animation.to_jshtml()))
print(f'it took {time.time()-start:.1f} seconds to run the code.')